In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3

In [ ]:
!pip install python-binance
!pip install binance-client

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
from get_data import crypto_data
from get_data import prepare_data

In [3]:
timeperiods = {}
timeperiods["BB"] = 10
timeperiods["EMA"] = 10
timeperiods["ADX0"] = 10
timeperiods["ADX0var"] = 10
timeperiods["ATR0"] = 10
timeperiods["ARO0"] = 10
timeperiods["WLR0"] = 10

In [ ]:
df = crypto_data(timeperiods)
df

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras

In [93]:
#test method
def test_params(timeperiods):
  df = crypto_data(timeperiods)
  print("======================================================")
  print("test for periods: ", timeperiods)
  example = df["df_BTC"]
  df_input = pd.DataFrame(columns = example.columns)
  for single in df.values():
    df_input = df_input.append(single)

  df_output = np.zeros(df_input.shape[0])
  for i in range(1, df_input.price.shape[0]):
    diff = df_input.price.iloc[i] - df_input.price.iloc[i-1]
    if diff > 0:
      df_output[i] = 1
    elif diff < 0:
      df_output[i] = -1

  df_input_test = df_input.iloc[::5]
  df_input_train = df_input.iloc[1::5]
  df_input_train = df_input_train.append(df_input.iloc[2::5])
  df_input_train = df_input_train.append(df_input.iloc[3::5])
  df_input_train = df_input_train.append(df_input.iloc[4::5])

  df_output_test = df_output[::5]
  df_output_train = df_output[1::5]
  df_output_train = np.concatenate((df_output_train, df_output[2::5]))
  df_output_train = np.concatenate((df_output_train, df_output[3::5]))
  df_output_train = np.concatenate((df_output_train, df_output[4::5]))


  clf = tree.DecisionTreeClassifier(max_depth=7)
  clf = clf.fit(df_input_train, df_output_train)
  preds = clf.predict(df_input_test)
  print("decision tree score test: ", 
        clf.score(df_input_test, df_output_test), 
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1))
  print("decision tree score train: ", clf.score(df_input_train, df_output_train))


  gnb = GaussianNB()
  gnb.fit(df_input_train, df_output_train)
  preds = gnb.predict(df_input_test)
  print("Gaussian NB score test: ", 
        gnb.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("Gaussian NB score train: ", gnb.score(df_input_train, df_output_train))

  gnb = MultinomialNB()
  gnb.fit(df_input_train, df_output_train)
  preds = gnb.predict(df_input_test)
  print("Multinomial NB score test: ", 
        gnb.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("Multinomial NB score train: ", gnb.score(df_input_train, df_output_train))



  clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
  clf.fit(df_input_train, df_output_train)
  preds = clf.predict(df_input_test)
  print("random forest score test: ", 
        clf.score(df_input_test, df_output_test),
        " up preds: ", np.sum(preds == 1),
        " down preds: ", np.sum(preds == -1),
        " 0 preds: ", np.sum(preds == -1)
  )
  print("random forest score train: ", clf.score(df_input_train, df_output_train))

  print("======================================================")
  return

  #model preparation
  ds_output_NN_train = tf.one_hot(df_output_train, depth = 3)
  ds_output_NN_test = tf.one_hot(df_output_test, depth = 3)

  model = tf.keras.models.Sequential([                    
    #tf.keras.layers.Conv1(filters= 2, kernel_size = 3, activation='tanh', use_bias=False),
    #tf.keras.layers.MaxPool2D(),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, use_bias=True, activation="relu", input_shape=(df_input.shape[1],)),
    tf.keras.layers.Dense(5, use_bias=True, activation="tanh"),
    tf.keras.layers.Dense(3, use_bias=True, activation="softmax")
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(0.0008),
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()],
  )

  model.fit(
      df_input_train,
      ds_output_NN_train,
      epochs=10,
      verbose = 0
  )

  print("NN score: ")
  model.evaluate(df_input_test, ds_output_NN_test, verbose=1)
  preds = np.argmax(model.predict(df_input_test), axis=1)
  print("NN score test: ", 
        " up preds: ", np.sum(preds == 2),
        " down preds: ", np.sum(preds == 0),
        " 0 preds: ", np.sum(preds == 1)
  )
  print("======================================================")

In [ ]:
for BB  in [2, 5, 10, 20]:
  for EMA  in [2, 5, 10, 20]:
    for ADX0  in [2, 5, 10, 20]:
      for ADX0var  in [2, 5, 10, 20]:
        for ATR0  in [2, 5, 10, 20]:
          for ARO0  in [2, 5, 10, 20]:
            for WLR0  in [2, 5, 10, 20]:
              
              timeperiods = {}
              timeperiods["BB"] = BB
              timeperiods["EMA"] = EMA
              timeperiods["ADX0"] = ADX0
              timeperiods["ADX0var"] = ADX0var
              timeperiods["ATR0"] = ATR0
              timeperiods["ARO0"] = ARO0
              timeperiods["WLR0"] = WLR0
              test_params(timeperiods)

test for periods:  {'BB': 2, 'EMA': 2, 'ADX0': 2, 'ADX0var': 2, 'ATR0': 2, 'ARO0': 2, 'WLR0': 2}
decision tree score test:  0.8398340248962656  up preds:  561  down preds:  606  0 preds:  606
decision tree score train:  0.8983191533513177
Gaussian NB score test:  0.12531120331950207  up preds:  48  down preds:  151  0 preds:  151
Gaussian NB score train:  0.12471467109358789
Multinomial NB score test:  0.23900414937759337  up preds:  7  down preds:  587  0 preds:  587
Multinomial NB score train:  0.247561734799751
random forest score test:  0.8506224066390041  up preds:  561  down preds:  644  0 preds:  644
random forest score train:  0.8588918862834612
test for periods:  {'BB': 2, 'EMA': 2, 'ADX0': 2, 'ADX0var': 2, 'ATR0': 2, 'ARO0': 2, 'WLR0': 5}
decision tree score test:  0.8605809128630706  up preds:  574  down preds:  590  0 preds:  590
decision tree score train:  0.9149201079062046
Gaussian NB score test:  0.12531120331950207  up preds:  48  down preds:  151  0 preds:  151
Gaussi

In [ ]:
for timeperiod  in [2, 5, 10, 20, 50, 100]:
  
  timeperiods = {}
  timeperiods["BB"] = timeperiod
  timeperiods["EMA"] = timeperiod
  timeperiods["ADX0"] = timeperiod
  timeperiods["ADX0var"] = timeperiod
  timeperiods["ATR0"] = timeperiod
  timeperiods["ARO0"] = timeperiod
  timeperiods["WLR0"] = timeperiod
  test_params(timeperiods)